In [2]:
!pip install -r requirements.txt
!pip install jupyter-lsp
!pip install 'python-lsp-server[all]'
!pip install "pymongo[srv]"
!%config Completer.use_jedi = False
!ln -s / .lsp_symlink

/bin/bash: line 0: fg: no job control
ln: failed to create symbolic link '.lsp_symlink/': File exists


In [2]:
import numpy as np
import nltk
import pandas as pd

from persistence import Connection as Database

In [3]:
from typing import List, Tuple

In [4]:
# load some data 
db: Database = Database()

Connected to DB


In [5]:
resumes = db.fetch_all_unlabeled_resumes()

In [6]:
from model.Job import Job

# Observations
- We observed that it/IT is a conflict with stop-word removal …
- Take the length of the experience into account 
- For example, statistical and statsitcs are close, maybe relevant to use embeddnings istead of intersection

In [18]:
print(len(resumes))

print(resumes[1]['interests'])

50
{'value': [{'value': 'Social Media', 'label': 0}, {'value': 'Spreading knowledge', 'label': 0}], 'label': 0}


In [8]:
import importlib
from model import keyword_utils
importlib.reload(keyword_utils)

<module 'model.keyword_utils' from '/home/jupyter/OpenCV/root/backend/model/keyword_utils.py'>

In [9]:

print(keyword_utils.get_section_texts_with_score_from_resume(resumes[0]))

[('Data Scientist \n ', 0.5909591785237429), ('Data Scientist @ A>RACE \n A>RACE is the Accenture Risk Analytics Center of Excellence. It is a center created within Accenture which deals with risk management projects with the objective to provide innovative IT solutions.', 0.9619381521900876), ('Stage Consulting @ A>RACE \n ', 0.7333717127785977), ("Tutor \n I was the tutor of the Master degree Direzione aziendale taught in the University of Bologna for one year. I managed the relations between professors and students, organized the schedules for the academic year and dealt with administrative issues related to the course's admission and graduation.", 0.03841496368395636), ('Mentee \n Mentee in the ICARO project. The project which last 9 months is an intensive entrepreneurial path aimed to bring university students close to the business culture. During this path, I learned to work together with students coming from different realities using a design thinking approach to solve real busi

In [11]:
import random
   
def flatten(l: List) -> List: 
    return [item for sublist in l for item in sublist] # flattening the resulting listprint(resume_text_base)
    
    
def train_keywords_for_job_title(job_title: str, dataset: List[dict]) -> Job: 
    """Trains the keyword scores for a job title given the dataset
    The dataset should be a list of labeled resumes, where the title of the labeling was the job title"""
    job: Job = Job(job_title)
    resume_text_base = [keyword_utils.get_section_texts_with_score_from_resume(resume) for resume in dataset]
    resume_text_base = flatten(resume_text_base)
    keywords = keyword_utils.extract_keywords_from_text([v[0] for v in resume_text_base])

    flat_keywords = []
    for i, row in enumerate(keywords): 
        for kw in row:
            flat_keywords.append(kw + (resume_text_base[i][1],))
    frame = pd.DataFrame(flat_keywords, columns=["word", "score", "paragraph_score"])

    frame['weighted_score'] = frame.score * frame.paragraph_score

    frame = frame.groupby('word', as_index=True).mean().sort_values(by=['weighted_score'], ascending=False)
    most_important = frame[frame.weighted_score > 0.2]

    m = most_important.weighted_score
    
    job.set_keywords(m.to_dict())

    
    return job

j = train_keywords_for_job_title("Data scientist", resumes)
    
    

In [12]:
print(len(j.keywords))

1190


In [15]:
from persistence.models import Resume

## Need to fix
 - Keyword extract funciton with keybert (Ø)
 - Embedding words (prebuilt) (K)
 - Distance func (K)
 
 - Training keyword/weights
 - word relevance to paragraph func (TF-IDF?)
 - Filter stopwords and stuff func 
 - Refactor scoring into class
 - Make a interface class for scoring a cv 
 - Function for cutting out content 



## Training
all cvs -> split into labeled-parts -> extract keywords (keybert) -> update score with label -> for each word output average -> List of words with score for position 


## Other features
 - Experience duration? 
 - How long ago? 

In [32]:
import model.resume_scorer
importlib.reload(model.resume_scorer)

scorer = model.resume_scorer.ResumeScorer()
out = scorer.score_resume(resumes[1], j)


In [33]:
out['experience']

{'value': [{'value': {'company': 'Google',
    'title': 'Startup Mentor',
    'location': 'Stockholm, Sweden',
    'fromDate': 'October 2022',
    'toDate': 'Present',
    'description': 'I am a mentor at Google’s Accelerator Mentor community.'},
   'label': 0.10979697031510403},
  {'value': {'company': 'AI4Diversity',
    'title': 'Director',
    'location': 'Stockholm, Sweden',
    'fromDate': 'December 2021',
    'toDate': 'Present',
    'description': '- I am a Director at AI4Diversity Sweden. - AI4Diversity is the fastest-growing nonprofit global initiative that engages and educates diverse communities about AI to benefit global society. - We believe AI makers need to be from diverse backgrounds to provide equal opportunities to everyone. - Our mission is to increase a more diverse representation and participation in AI. - We are a community-driven initiative bringing empowerment, knowledge, and active collaboration via education, research, events, and blogging.'},
   'label': 0.0

In [ ]:
from datetime import datetime


# this is just for Karl to learn, can be removed later
# for words like "today" or "present" use today()

fromDate = first['experience']['value'][1]['value']['fromDate']
print(fromDate)
datetime_object_from = datetime.strptime(fromDate, '%B %Y')
print(datetime_object_from)

toDate = first['experience']['value'][1]['value']['toDate']
print(toDate)
datetime_object_to = datetime.strptime(toDate, '%B %Y')
print(datetime_object_to)

print(datetime_object_to-datetime_object_from)

today = datetime.today()
print(today)

diff = today - datetime_object_to
days = np.floor(diff.total_seconds()/86400)
print(int(days)) # gets the number of days

November 2019
2019-11-01 00:00:00
September 2021
2021-09-01 00:00:00
670 days, 0:00:00
2022-11-09 06:24:09.355882
434


In [ ]:
def get_length_of_exp(experience) -> int:
    


670 days, 0:00:00
